In [ ]:
from BiopTort import compute_snake, \
    calculate_grade, \
    filter_contours, \
    remove_loops, \
    interpolate_points, \
    remove_background_points, \
    apply_gaussian_filter
    
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
image_num = '_1687428'
fn = f'/media/jackson/backup/dp_data/tortuosity_study/tortuosity_training_set/{image_num}.svs'
mask_filepath = f'/media/jackson/backup/dp_data/tortuosity_study/QC_results/training_set_final_run/usable_tissue_masks/{image_num}.svs_mask_use.png'
savedir = f'./algorithm_demo_output/{image_num}'

In [ ]:
if not os.path.exists(savedir):
    os.mkdir(savedir)
else:
    print('Directory already exists.')

In [ ]:
snake, img, contours, rp, b, num_points = compute_snake(fn, mask_path=mask_filepath)

In [ ]:
# display contours
fmts = ['c', '#ffa530', 'y', 'k', 'm', '#918211','#e875fa', '#b5d9ff', '#ffb5bb', '#3e754d', '#0014c7']
# plt.imshow(b)
for i, c in enumerate(contours):
    plt.plot(c.T[1],-1 * c.T[0], fmts[i % len(fmts)], label=f'contour {i + 1}')
    # plt.fill(c.T[1], c.T[0], facecolor=f'white', alpha=1, label=f'contour {i + 1}')

plt.axis('equal')
plt.axis('off')
plt.legend(fontsize=6)
plt.savefig(f'{savedir}/contours.png', dpi=600)


In [ ]:
# display contours
fmts = ['c', '#ffa530', 'y', 'k', 'm', '#918211','#e875fa', '#b5d9ff', '#ffb5bb', '#3e754d', '#0014c7']
# plt.imshow(b)
for i, c in enumerate(contours):
    plt.plot(c.T[1],-1 * c.T[0], fmts[i % len(fmts)], label=f'contour {i + 1}')
    # plt.fill(c.T[1], c.T[0], facecolor=f'white', alpha=1, label=f'contour {i + 1}')

plt.axis('equal')
plt.axis('off')
plt.legend(fontsize=6)

plt.plot(snake[:, 1], -1 * snake[:, 0], 'r', label='snake')
plt.savefig(f'{savedir}/contours_snake.png', dpi=600)

In [ ]:
# ------------- REMOVE LOOPS -------------
snake = remove_loops(snake)
# ------------- SNAKE SMOOTHING -------------
snake = interpolate_points(snake, num_points)
snake = remove_background_points(snake, b)
snake = apply_gaussian_filter(snake, sigma=20)

snake = snake[0:-1:10]

# display contours
fmts = ['c', '#ffa530', 'y', 'k', 'm', '#918211','#e875fa', '#b5d9ff', '#ffb5bb', '#3e754d', '#0014c7']
# plt.imshow(b)
for i, c in enumerate(contours):
    plt.plot(c.T[1],-1 * c.T[0], fmts[i % len(fmts)], label=f'contour {i + 1}')
    # plt.fill(c.T[1], c.T[0], facecolor=f'white', alpha=1, label=f'contour {i + 1}')

plt.axis('equal')
plt.axis('off')
plt.legend(fontsize=6)
plt.plot(snake[:, 1], -1 * snake[:, 0], 'r', label='snake')
plt.savefig(f'{savedir}/contours_snake_smoothed.png', dpi=600)

In [ ]:
# ------------- CALCULATE XY TORTUOSITY -------------
L = np.sum(np.linalg.norm(np.diff(snake, axis=0), axis=-1))
L_cumu = np.cumsum(np.linalg.norm(np.diff(snake, axis=0), axis=-1))
L_0 = np.linalg.norm(snake[-1] - snake[0])
xy_tortuosity = round(L / L_0, 4)
# --------------------------------------------------------

# calculate two boundaries. One at the 10th percentile of the snake and one at the 90th percentile.
beginning = None
end = (500,500)

for i in range(len(snake)):
    if beginning is None and L_cumu[i] > L * 0.1:
        beginning = snake[i]
    if L_cumu[i] > L * 0.9:
        end = snake[i]
        break
vect = np.array([np.sin(-1 * rp.orientation), np.cos(rp.orientation)])
bound1 = np.array([beginning - vect * 100, beginning + vect * 100])
bound2 = np.array([end - vect * 100, end + vect * 100])

print(f'bound1: {bound1}')

# filter contours
filtered_contours = filter_contours(contours, (beginning, end), rp.orientation, snake)

# ------------- CALCULATE GRADE -------------
b_gap_count = len(filtered_contours) - 1
pred_grade = calculate_grade(xy_tortuosity, b_gap_count)
print(f'xy_tortuosity: {xy_tortuosity}')
# --------------------------------------------------------


# create plot
plt.imshow(np.ones((3, img.size[0], img.size[1])).T * 255)
fmts = ['c', '#ffa530', 'y', 'k', 'm', '#918211','#e875fa', '#b5d9ff', '#ffb5bb', '#3e754d', '#0014c7']
print(f'# of filtered contours: {len(filtered_contours)}')
for i, c in enumerate(filtered_contours):
    plt.plot(c.T[1],c.T[0], fmts[i % len(fmts)], label=f'contour {i + 1}')

    # plt.fill(c.T[1], c.T[0], facecolor=f'{fmts[i]}', alpha=0.6, label=f'contour {i + 1}')
    

plt.scatter(snake[:, 1], snake[:, 0], marker='+', label='L', c='r', s=1)
# plt.plot(snake[:, 1], snake[:, 0], label='L', c='r', lw=1)
plt.plot([snake[0,1], snake[-1,1]], [snake[0,0], snake[-1,0]], '--b', lw=1, label='L_0')
plt.plot(bound1[:, 1], bound1[:, 0], 'g', lw=1, label='10th percentile of L')
plt.plot(bound2[:, 1], bound2[:, 0], 'g', lw=1, label='90th percentile of L')
plt.legend(fontsize=6)
plt.axis('off')
plt.savefig(f'{savedir}/contours_snake_smoothed_filtered.png', dpi=600)

In [ ]:
# create plot
plt.imshow(img)
fmts = ['c', '#ffa530', 'y', 'k', 'm', '#918211','#e875fa', '#b5d9ff', '#ffb5bb', '#3e754d', '#0014c7']
print(f'# of filtered contours: {len(filtered_contours)}')
for i, c in enumerate(filtered_contours):
    # plt.plot(c.T[1], c.T[0], f'-.',
    #          c=f'{fmts[i]}', lw=1, label=f'contour {i}')

    plt.fill(c.T[1], c.T[0], facecolor=f'{fmts[i]}', alpha=0.6, label=f'contour {i + 1}')
    

plt.scatter(snake[:, 1], snake[:, 0], marker='+', label='L', c='r', s=1)
# plt.plot(snake[:, 1], snake[:, 0], label='L', c='r', lw=1)
plt.plot([snake[0,1], snake[-1,1]], [snake[0,0], snake[-1,0]], '--b', lw=1, label='L_0')
plt.plot(bound1[:, 1], bound1[:, 0], 'g', lw=1, label='10th percentile of L')
plt.plot(bound2[:, 1], bound2[:, 0], 'g', lw=1, label='90th percentile of L')
plt.legend(fontsize=6)
plt.savefig(f'{savedir}/contours_snake_smoothed_filtered.png', dpi=600)